**Inferenced**

In [2]:
!python inference.py -m samples/5eed8296d6b96_60dac2a791cc1.flac -c configs/train_rnn.yml -w weights/__DualPath_RNN_179_-3.1895.pt -s samples

saved in: samples/DualPath_RNN/spk1/60dac2a791cc1.flac
saved in: samples/DualPath_RNN/spk2/5eed8296d6b96.flac


**Ploting loss**

In [ ]:
from utils.plot import save_graph_tb_log_metrics

save_graph_tb_log_metrics(
    first_csv_path='checkpoints/train_rnn/Loss_Train.csv',
    second_csv_path='checkpoints/train_rnn/Loss_Validation.csv',
    name_ox='Epoch',
    name_oy='Loss',
    loc = 'upper right',
    pth_save='pics/Loss_Train_Val.png'
)

**Evaluation**

In [1]:
import argparse
import sys

import torch
from torchmetrics.audio import PermutationInvariantTraining as PIT
from torchmetrics.functional.audio import signal_distortion_ratio as sdr
from torchmetrics.functional.audio import scale_invariant_signal_noise_ratio as sisnr
# from torchmetrics.audio import ScaleInvariantSignalNoiseRatio as sisnr
from tqdm.notebook import tqdm

# from losses import sisnr_pit
from utils.load_config import load_config  
from models import MODELS
from data.DiarizationDataset import DiarizationDataset

parser = argparse.ArgumentParser()
parser.add_argument("-p", "--hparams", type=str, default="./configs/dev_dualpathrnn.yml", help="hparams config file")
args, unknown = parser.parse_known_args()  
cfg = load_config(args.hparams)

**Dualpath-RNN**

In [2]:
model_class = MODELS[cfg['xp_config']['model_type']]
model = model_class(**cfg['model'])
device = cfg['trainer']['device']
model.to(device)
datamodule = DiarizationDataset(**cfg['data']).setup(stage = 'eval')
test_dataloader = datamodule.test_dataloader()

Size of test set: 848
Elapsed time 'setup': 00:00:00.70


In [3]:
weight = './weights/__DualPath_RNN_179_-3.1895.pt'
dicts = torch.load(weight, map_location=device, weights_only=False)
model.load_state_dict(dicts['model_state_dict'])

model.eval()
running_sdr = 0.0
running_sisnr = 0.0
pit_sdr = PIT(sdr).to(device)
pit_sisnr = PIT(sisnr).to(device)

In [4]:
for inputs, labels in tqdm(test_dataloader):
    '''
    batch = 1 spk = 2 time = 3200
    inputs: [batch, time] 
    outputs and labels: [torch.randn(batch, time) for _ in range(spk)]  
    expectention outputs and labels for torch audio-loss: torch.Size([batch, spk, time])
    '''
    inputs, labels = inputs.to(device), [l.to(device) for l in labels]
    with torch.no_grad():
        outputs = [s.detach() for s in model(inputs)]
        labels = torch.stack(labels, dim=1).to(device)
        outputs = torch.stack(outputs, dim=1).to(device)
        pit_sdr.update(outputs, labels)
        pit_sisnr.update(outputs, labels)
        
print('sdr', pit_sdr.compute().item())
print('sisnr', pit_sisnr.compute().item())

  0%|          | 0/848 [00:00<?, ?it/s]

sdr 5.21800422668457
sisnr 4.563263416290283


In [5]:
# sdr -2.0657341480255127
# sdr second 5.21800422668457
# sisnr -4.563594581563693

In [ ]:
# sdr 5.21800422668457
# sisnr 4.563263416290283